<a href="https://colab.research.google.com/github/amalianuruljannah/TugasSosmed1/blob/main/Tugas_Sosmed_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from google.colab import files
from urllib.parse import urljoin

# Target URL CNN Indonesia
URL = "https://www.cnnindonesia.com/nasional"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(URL, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

data = []
articles = soup.find_all("article")

for art in articles:
    title_tag = art.find("h2")
    link_tag = art.find("a")
    summary_tag = art.find("p")
    time_tag = art.find("span", class_="date")
    category_tag = art.find("span", class_="kanal")

    title = title_tag.get_text(strip=True) if title_tag else None
    link = link_tag["href"] if link_tag else None
    summary = summary_tag.get_text(strip=True) if summary_tag else None
    publish_time = time_tag.get_text(strip=True) if time_tag else None
    category = category_tag.get_text(strip=True) if category_tag else "Nasional"

    # --- Normalisasi link ---
    if link and not link.startswith("http"):
        link = urljoin(URL, link)  # gabungkan dengan base URL

    # --- Extra step: buka halaman berita untuk ambil isi & penulis ---
    author = None
    content = None
    tags = None
    if link and link != "#":
        try:
            detail = requests.get(link, headers=headers, timeout=10)
            soup_detail = BeautifulSoup(detail.text, "html.parser")

            # Cari penulis
            author_tag = soup_detail.find("div", class_="author")
            if author_tag:
                author = author_tag.get_text(strip=True)

            # Cari isi berita
            content_tags = soup_detail.find_all("p")
            content = " ".join([c.get_text(strip=True) for c in content_tags]) if content_tags else None

            # Cari tags (kalau ada)
            tag_section = soup_detail.find("div", class_="tag-list")
            if tag_section:
                tags = [t.get_text(strip=True) for t in tag_section.find_all("a")]
        except Exception as e:
            print(f"Gagal ambil detail {link}: {e}")

    if title and link:
        data.append({
            "title": title,
            "link": link,
            "summary": summary,
            "publish_time": publish_time,
            "category": category,
            "author": author,
            "content": content,
            "tags": tags
        })

# Konversi ke DataFrame
df = pd.DataFrame(data)

# --- EXPORT FILE ---

# CSV
csv_file = "berita.csv"
df.to_csv(csv_file, index=False, encoding="utf-8")

# Excel
excel_file = "berita.xlsx"
df.to_excel(excel_file, index=False)

# JSON
json_file = "berita.json"
with open(json_file, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

# --- DOWNLOAD FILE DARI COLAB ---
files.download(csv_file)
files.download(excel_file)
files.download(json_file)

print(f"Scraping selesai , total {len(data)} berita dengan metadata lengkap berhasil disimpan!")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Scraping selesai , total 28 berita dengan metadata lengkap berhasil disimpan!
